In [226]:
import numpy as np 
import matplotlib.pyplot as plt 
from tomography import Random_Measurements, NearSparseTomography_v2
from qiskit_aer.primitives import Estimator
from qiskit_aer import AerSimulator 
from qiskit.quantum_info import Statevector, random_clifford, state_fidelity
from qiskit_aer.noise import NoiseModel, depolarizing_error 

In [227]:
error_1 = depolarizing_error( 0.01, 1 )
error_2 = depolarizing_error( 0.02, 2 )
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['h', 's', 'sd', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

In [228]:
NQ = 3

backend = Estimator(backend_options={'shots':1e3, 
                                        'method':"stabilizer",
                                        'noise_model':noise_model},
                                transpile_options={'optimization_level':0},
                                ) 

In [230]:
Omega = random_clifford( NQ ).to_circuit()
OmegaS = np.array( Statevector(Omega) )
OmegaM = np.outer( OmegaS, OmegaS.conj() )
OmegaM.round(2) 

array([[ 0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,
        -0.12-0.j, -0.12-0.j],
       [ 0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,
        -0.12-0.j, -0.12-0.j],
       [ 0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,
        -0.12-0.j, -0.12-0.j],
       [ 0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,
        -0.12-0.j, -0.12-0.j],
       [ 0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,
        -0.12-0.j, -0.12-0.j],
       [ 0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,  0.12+0.j,
        -0.12-0.j, -0.12-0.j],
       [-0.12-0.j, -0.12-0.j, -0.12-0.j, -0.12-0.j, -0.12-0.j, -0.12-0.j,
         0.12+0.j,  0.12+0.j],
       [-0.12-0.j, -0.12-0.j, -0.12-0.j, -0.12-0.j, -0.12-0.j, -0.12-0.j,
         0.12+0.j,  0.12+0.j]])

In [231]:
qasm = AerSimulator(method='density_matrix',
                        noise_model=noise_model)

Omega_density = Omega.copy()
Omega_density.save_density_matrix()
OmegaM_noisy = np.array( qasm.run(Omega_density).result().data()['density_matrix'] )
OmegaM_noisy.round(2) 

array([[ 0.12+0.j,  0.1 -0.j,  0.1 +0.j,  0.1 +0.j,  0.1 +0.j,  0.11+0.j,
        -0.1 -0.j, -0.1 +0.j],
       [ 0.1 +0.j,  0.12-0.j,  0.1 +0.j,  0.1 -0.j,  0.11-0.j,  0.1 +0.j,
        -0.1 -0.j, -0.1 +0.j],
       [ 0.1 -0.j,  0.1 -0.j,  0.12-0.j,  0.1 -0.j,  0.1 +0.j,  0.1 +0.j,
        -0.1 +0.j, -0.11+0.j],
       [ 0.1 -0.j,  0.1 +0.j,  0.1 -0.j,  0.12+0.j,  0.1 -0.j,  0.1 +0.j,
        -0.11-0.j, -0.1 +0.j],
       [ 0.1 -0.j,  0.11+0.j,  0.1 +0.j,  0.1 +0.j,  0.12-0.j,  0.1 +0.j,
        -0.1 +0.j, -0.1 +0.j],
       [ 0.11-0.j,  0.1 -0.j,  0.1 -0.j,  0.1 +0.j,  0.1 -0.j,  0.12+0.j,
        -0.1 +0.j, -0.1 -0.j],
       [-0.1 +0.j, -0.1 +0.j, -0.1 -0.j, -0.11+0.j, -0.1 -0.j, -0.1 -0.j,
         0.12+0.j,  0.1 -0.j],
       [-0.1 -0.j, -0.1 +0.j, -0.11-0.j, -0.1 -0.j, -0.1 -0.j, -0.1 +0.j,
         0.1 +0.j,  0.12-0.j]])

In [232]:
RM = Random_Measurements(NQ)
RM.RandomMeasurements( 4**(NQ-1), Omega, backend ) 

In [233]:
phi_in = OmegaS 
phi_out = NearSparseTomography_v2( phi_in, RM ) 
rho_out = phi_out@phi_out.T.conj()  
rho_out = rho_out / np.trace( rho_out ) 
np.vdot( OmegaS, rho_out@OmegaS ).real

0.07855388206612793


0.40826372511313513

In [234]:
phi_in = 0.001*( np.random.randn(2**NQ,4) 
                + 1j*np.random.randn(2**NQ,4) )
phi_in[:,0] = OmegaS 
phi_out = NearSparseTomography_v2( phi_in, RM ) 
rho_out = phi_out @ phi_out.T.conj()  
rho_out = rho_out / np.trace( rho_out ) 
np.vdot( OmegaS, rho_out@OmegaS ).real 

0.0034329681910460316


0.22443035066072625

In [235]:
phi_in.round(2) 

array([[-0.25+0.25j, -0.  +0.j  , -0.  +0.j  , -0.  +0.j  ],
       [-0.25+0.25j, -0.  -0.j  , -0.  -0.j  , -0.  +0.j  ],
       [-0.25+0.25j, -0.  -0.j  ,  0.  -0.j  , -0.  +0.j  ],
       [-0.25+0.25j,  0.  +0.j  ,  0.  +0.j  , -0.  +0.j  ],
       [-0.25+0.25j, -0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ],
       [-0.25+0.25j,  0.  +0.j  ,  0.  -0.j  ,  0.  -0.j  ],
       [ 0.25-0.25j, -0.  -0.j  , -0.  -0.j  ,  0.  +0.j  ],
       [ 0.25-0.25j, -0.  +0.j  ,  0.  -0.j  , -0.  -0.j  ]])

In [236]:
phi_out.round(2)

array([[-0.39+0.54j, -0.49-0.16j, -0.41-0.24j, -0.51+0.24j],
       [-0.25+0.37j, -0.44-0.6j , -0.12-0.13j, -0.38+0.13j],
       [-0.29+0.26j,  0.37+0.35j,  0.46-0.02j, -0.2 +0.52j],
       [-0.57+0.21j,  0.52+0.31j,  0.33-0.01j, -0.14+0.48j],
       [-0.59+0.14j, -0.41+0.34j,  0.26-0.15j,  0.37-0.39j],
       [-0.14+0.38j, -0.37+0.35j,  0.36-0.36j,  0.54-0.47j],
       [ 0.19-0.32j, -0.18+0.11j,  0.31-0.68j, -0.15+0.38j],
       [ 0.32-0.43j, -0.26+0.13j,  0.23-0.76j, -0.21+0.24j]])

In [237]:
state_fidelity( rho_out, OmegaM )

0.22443035401129585

In [238]:
state_fidelity( rho_out, OmegaM_noisy )

0.4457243144743832